In [37]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [38]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-0n9_6szo
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-0n9_6szo


In [39]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [40]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [41]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [42]:
#GPU 사용
device = torch.device("cuda:0")

In [43]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [44]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
# data loading 
import pandas as pd
data_first = pd.read_csv('/content/drive/Othercomputers/내 컴퓨터/cyber-bullying-detect/nlp/data/hate_speech_binary_dataset.csv')
data_second = pd.read_csv('/content/drive/Othercomputers/내 컴퓨터/cyber-bullying-detect/nlp/data/data2.tsv',sep='\t')

data_second = data_second.drop(['contain_gender_bias','bias'],axis = 1)
data_second.columns = ['문장', '혐오 여부']
data_second.loc[(data_second['혐오 여부'] == "hate"), '혐오 여부'] = 0
data_second.loc[(data_second['혐오 여부'] == "none"), '혐오 여부'] = 1
data_second.loc[(data_second['혐오 여부'] == "offensive"), '혐오 여부'] = 0
data = pd.concat([data_first, data_second], ignore_index = True)

In [46]:
data_list = []
for q, label in zip(data['문장'],data['혐오 여부']) :
  data = []
  data.append(str(q))
  data.append(str(label))

  data_list.append(data)

In [47]:
# 데이터 숫자 조절 - > 모델 제대로 할때는 삭제
#data_list = data_list[:1000]

In [48]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.25)

In [49]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [50]:
# Setting parameters 조절 열심히하자 
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
for i in dataset_train:
  if i[1] == 'offensive':
    print(i)

In [51]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

using cached model


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [52]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [53]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [54]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.6604505777359009 train acc 0.578125
epoch 1 batch id 201 loss 0.37836185097694397 train acc 0.675761815920398
epoch 1 batch id 401 loss 0.16337677836418152 train acc 0.7887702618453866
epoch 1 batch id 601 loss 0.13789334893226624 train acc 0.8321287437603994
epoch 1 batch id 801 loss 0.1391882300376892 train acc 0.8558247503121099
epoch 1 batch id 1001 loss 0.3075261414051056 train acc 0.8703796203796204
epoch 1 batch id 1201 loss 0.1458231508731842 train acc 0.8810496461282264
epoch 1 batch id 1401 loss 0.27691781520843506 train acc 0.8887178800856531
epoch 1 batch id 1601 loss 0.09778602421283722 train acc 0.8944800124921923
epoch 1 batch id 1801 loss 0.25575411319732666 train acc 0.8991879511382566
epoch 1 batch id 2001 loss 0.10129851847887039 train acc 0.9033842453773113
epoch 1 batch id 2201 loss 0.09191212803125381 train acc 0.906647546569741

epoch 1 train acc 0.9086206896551724


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.9351582687338501


...


epoch 3 test acc 0.9522771317829457


epoch 4 batch id 1 loss 0.03268382325768471 train acc 0.984375
epoch 4 batch id 201 loss 0.1669662743806839 train acc 0.982431592039801
epoch 4 batch id 401 loss 0.014293454587459564 train acc 0.9829722568578554
epoch 4 batch id 601 loss 0.013404060155153275 train acc 0.9839850249584027
epoch 4 batch id 801 loss 0.011197359301149845 train acc 0.9847066167290887
epoch 4 batch id 1001 loss 0.17164184153079987 train acc 0.9854676573426573
epoch 4 batch id 1201 loss 0.004008210264146328 train acc 0.9860142589508742
epoch 4 batch id 1401 loss 0.15937866270542145 train acc 0.9863601891506067
epoch 4 batch id 1601 loss 0.001298554358072579 train acc 0.9867563241723922
epoch 4 batch id 1801 loss 0.09467849135398865 train acc 0.9872119655746807
epoch 4 batch id 2001 loss 0.009165855124592781 train acc 0.9874515867066467
epoch 4 batch id 2201 loss 0.001216395408846438 train acc 0.9875553725579282

epoch 4 train acc 0.9876212284482758



epoch 4 test acc 0.950359334625323


epoch 5 batch id 1 loss 0.03510977327823639 train acc 0.984375
epoch 5 batch id 201 loss 0.06743944436311722 train acc 0.9904384328358209
epoch 5 batch id 401 loss 0.002682193648070097 train acc 0.9909600997506235
epoch 5 batch id 601 loss 0.002402057172730565 train acc 0.9919665141430949
epoch 5 batch id 801 loss 0.0006166385719552636 train acc 0.9921972534332085
epoch 5 batch id 1001 loss 0.1636294573545456 train acc 0.9927260239760239
epoch 5 batch id 1201 loss 0.005006151739507914 train acc 0.9927534346378019
epoch 5 batch id 1401 loss 0.1359802633523941 train acc 0.9928622412562456
epoch 5 batch id 1601 loss 0.0011462480761110783 train acc 0.9931390537164272
epoch 5 batch id 1801 loss 0.0891532152891159 train acc 0.9933977651304831
epoch 5 batch id 2001 loss 0.001327196485362947 train acc 0.9934720139930034
epoch 5 batch id 2201 loss 0.0006165289087221026 train acc 0.9935824625170377

epoch 5 train acc 0.9936422413793103



epoch 5 test acc 0.9523578811369509


In [55]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("혐오가")
            elif np.argmax(logits) == 1:
                test_eval.append("none")


        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

using cached model


In [56]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 개씨발년들아


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 안녕하세요 김현우 입니다
>> 입력하신 내용에서 none 느껴집니다.




KeyboardInterrupt: ignored

In [58]:
PATH = '/content/drive/Othercomputers/내 컴퓨터/cyber-bullying-detect/nlp/'
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')

In [59]:
torch.save(model, PATH + 'model.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')

In [61]:
#test 단계

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

model = torch.load(PATH+'model.pt')
model.to(device)
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("혐오가")
            elif np.argmax(logits) == 1:
                test_eval.append("none")


        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

using cached model
